Extract all bid csv from the page: https://bidarchive.relayscan.io/ethereum/mainnet/2024-06/index.html

In [5]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
import re

# URL of the page containing the ZIP file links
index_url = "https://bidarchive.relayscan.io/ethereum/mainnet/2024-06/index.html"

# Fetch the content of the page
response = requests.get(index_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract all links that match the pattern
zip_urls = []
for link in soup.find_all('a', href=True):
    href = link['href']
    if re.match(r'\d{4}-\d{2}-\d{2}_all\.csv\.zip$', href):
        zip_urls.append(index_url.rsplit('/', 1)[0] + '/' + href)

def download_and_extract_csv(zip_url):
    response = requests.get(zip_url)
    with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
        csv_files = [file for file in thezip.namelist() if file.endswith('.csv')]
        frames = [pd.read_csv(thezip.open(file)) for file in csv_files]
        return pd.concat(frames, ignore_index=True)

# Combine all data into a single DataFrame
all_data = pd.concat([download_and_extract_csv(url) for url in zip_urls], ignore_index=True)

# Display the DataFrame
print(all_data.shape[0])


/var/folders/mf/lxpn1ltx7t97r1v1k92vly1w0000gn/T/ipykernel_99915/2289705463.py:26: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(thezip.open(file)) for file in csv_files]
/var/folders/mf/lxpn1ltx7t97r1v1k92vly1w0000gn/T/ipykernel_99915/2289705463.py:26: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(thezip.open(file)) for file in csv_files]
/var/folders/mf/lxpn1ltx7t97r1v1k92vly1w0000gn/T/ipykernel_99915/2289705463.py:26: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(thezip.open(file)) for file in csv_files]
/var/folders/mf/lxpn1ltx7t97r1v1k92vly1w0000gn/T/ipykernel_99915/2289705463.py:26: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(thezip.open(file)) for file in csv_files]
/var

175115767


In [18]:
from data_prep_utils import *
from plot_prep_utils import *

pd.set_option('display.float_format', lambda x: '%.2f' % x)

df_builder = get_builder_info_from_latest_TLDR_talk()
df_eigenphi_builder = df_builder[df_builder['builder_label'] == 'EigenPhi']

eigenphi_bids = all_data[all_data['builder_pubkey'].isin(df_eigenphi_builder['builder_pubkey'])]


eigenphi_bids['value'] = pd.to_numeric(eigenphi_bids['value'], errors='coerce')
print(eigenphi_bids.shape[0])
print(eigenphi_bids['value'].describe()/10**18)
percentile_95 = eigenphi_bids['value'].quantile(0.95)
print(percentile_95/10**18)
percentile_99 = eigenphi_bids['value'].quantile(0.99)
print(percentile_99/10**18)

1272084
count    0.00
mean     0.02
std      0.12
min      0.00
25%      0.01
50%      0.01
75%      0.02
max     23.98
Name: value, dtype: float64
0.0610512460884119
0.17786455978862215


/var/folders/mf/lxpn1ltx7t97r1v1k92vly1w0000gn/T/ipykernel_99915/2844635042.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eigenphi_bids['value'] = pd.to_numeric(eigenphi_bids['value'], errors='coerce')
